In [1]:
# Import Libraries

import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import pickle

In [2]:
# Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
#Load test data
root = "D:/Kaggle/Energy Prediction/Data/"
df_test = pd.read_csv(f"{root}test.csv")
df_test = df_test.drop(columns=['row_id'])
building_metadata = pd.read_csv(f"{root}building_metadata.csv")
weather_test = pd.read_csv(f"{root}weather_test.csv")
sample = pd.read_csv(f"{root}sample_submission.csv")
df_test = reduce_mem_usage(df=df_test)
weather_test = reduce_mem_usage(df=weather_test)
df_test = df_test.merge(building_metadata, on='building_id', how='left')
df_test = df_test.merge(weather_test, on=['site_id', 'timestamp'], how='left')

Mem. usage decreased to 437.43 Mb (54.2% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)


In [4]:
#Data engineering
df_test.primary_use = LabelEncoder().fit_transform(df_test.primary_use)
df_test['timestamp'] = pd.to_datetime(arg=df_test['timestamp'])
df_test['year'] = df_test['timestamp'].dt.year
df_test['month'] = df_test['timestamp'].dt.month
df_test['day'] = df_test['timestamp'].dt.day
df_test['hour'] = df_test['timestamp'].dt.hour
df_test["weekday"] = df_test['timestamp'].dt.weekday
drop_features = ['day', 'month', 'year', 'cloud_coverage', 'precip_depth_1_hr', 'sea_level_pressure', 
        'wind_direction', 'wind_speed', 'floor_count', 'year_built']
df_test.drop(drop_features, axis=1, inplace=True)
# Missing value handling
cols = ['air_temperature','dew_temperature']
df_test.loc[:, cols] = df_test.loc[:, cols].interpolate(axis=0)
#Drop timestamp
drop_features = ["timestamp"]
df_test.drop(drop_features, axis=1, inplace=True)
x_test = df_test.values

In [5]:
df_test.isnull().sum()

building_id        0
meter              0
site_id            0
primary_use        0
square_feet        0
air_temperature    0
dew_temperature    0
hour               0
weekday            0
dtype: int64

In [6]:
df_test.to_csv('altered_test.csv', index=False)